In [1]:
from deep_translator import GoogleTranslator

from tqdm.auto import tqdm
import pandas as pd
from datasets import load_from_disk

import json
import os
import re

In [11]:
def google_translate(sent):
    ko_to_en = GoogleTranslator(source='auto', target='en')
    en_to_ko = GoogleTranslator(source='auto', target='ko')
    try:
        en_sent = ko_to_en.translate(sent)
        ko_sent = en_to_ko.translate(en_sent)
    except:
        return sent
    
    return ko_sent


def translate_bulk(sent_list):
    new_sent = []
    for idx in tqdm(range(len(sent_list)), desc="Tons of Translations Ongoing..."):
        trans_sent = google_translate(sent_list[idx])
        new_sent.append(trans_sent)
    return new_sent

def translate_sentence(sentence):
    trans_sent = google_translate(sentence)
    return trans_sent

In [3]:
train_dataset = load_from_disk("../data/train_dataset/")['train']

In [4]:
train_dataset.num_rows

3952

In [5]:
context = list(train_dataset['context'])
question = list(train_dataset['question'])
answers = list(train_dataset['answers'])

In [6]:
answers[0]

{'answer_start': [235], 'text': ['하원']}

In [7]:
def match_answer(sent, answer):
    for idx in range(len(sent)):
        if sent[idx:idx+len(answer)] == answer:
            return idx


def proper_translate(corpus, answer_dict):

    sentences = corpus.split('.')
    sentences = [(sent+'.', len(sent)+1) for sent in sentences]
    reference = [False for _ in range(len(sentences))]
    new_corpus = []
    answer, start_idx = answer_dict['text'][0], answer_dict['answer_start'][0]
    idx_count, new_idx = 0, 0
    
    for idx, sent in enumerate(sentences):
        if start_idx in range(idx_count, idx_count+sent[1]) and answer in sent[0]:
            new_idx += match_answer(sent[0], answer)
            reference[idx] = True
            break
        idx_count += sent[1]

    idx_accumulate = True
    for idx in range(len(reference)):
        if reference[idx] is True:
            new_corpus.append(sentences[idx][0]+' ')
            idx_accumulate = False
        else:
            trans_sent = translate_sentence(sentences[idx][0])
            new_corpus.append(trans_sent+' ')
            if idx_accumulate:
                new_idx += len(trans_sent)+1
        
        
    
    return ''.join(new_corpus)[:-3], new_idx

In [12]:
IDX = 199
new_text, new_id = proper_translate(context[IDX], answers[IDX])
new_text, new_id

('1338년부터 1343년까지 5년간 추판왕조의 인주왕조 내부 갈등은 복잡했다. 내전은 1338년 자이르 왕조의 신탁인 마수드 샤가 돌아와 갸스 알딘과 맞서면서 시작되었습니다. 1338년(또는 1339년) 기야스 알딘이 죽고 마수드 샤의 승리로 둘 사이의 갈등은 끝이 났지만 인주 왕조의 내란은 여기서 그치지 않았다. 샤라프 알딘의 넷째 아들인 샴스 알딘은 추판 왕조의 피르 호세인과 힘을 합쳐 쉬라즈를 침공했다. Masud Shah는 북쪽으로 달아났고 Shams al-Din은 Inju 왕조의 통치자가 되었습니다. 그러나 Shams Aldin은 Fir Hossain에게 배신당해 살해당했고, Fir Hossain도 쉬라즈 내에서 대중의 관심을 잃어 쫓겨났다. \\n\\n호사인 전나무는 이듬해 군대를 이끌고 돌아와 쉬라즈를 점령했다. 그러나 이번에는 Sharaf Al-Din의 아들인 Abu Ishaq에 의해 곧 전복되었습니다. Abu Ishaq는 또한 Shiraz를 오랫동안 점령하지 않았고 Chupan 왕조의 Yagi Basti와 연합한 Masud Shah에 의해 쫓겨났습니다. 그러나 쉬라즈가 함락된 후 둘 사이에 갈등이 생겨 결국 야기 바스티가 마수드 샤를 죽였다. Masoud Shah를 살해한 Yagi Basti도 쉬라즈에서 지지를 잃었고 1343년 Abu Ishak에 의해 축출되었습니다.  아부 이샤크에게 쫓겨난 야기 바스티는 조카인 마렉 아시라프와 힘을 합쳐 시라즈로 다시 쳐들어왔으나 1343년 12월에 추판 왕조의 지배자인 하산 쿠첵이 죽었다는 소식을 듣고 추판 왕조에서의 종주권을 차지하기 위해 본국으로 회군하였다. 이후 1343년부터 1357년까지 인주 왕조의 유일한 통치자로 아부 이샤크가 통치하였다.',
 697)

In [13]:
question[IDX], context[IDX], answers[IDX]['answer_start'][0], answers[IDX]['text'][0]

('야기 바스티의 시라즈 침략을 도운 사람은?',
 '인주 왕조의 내분은 1338년부터 1343년까지 5년간 추판 왕조의 왕족들과 얽혀 복잡하게 진행되었다. 내분은 1338년에 자라이르 왕조에 의탁하고 있던 마수드 샤가 돌아와 기야스 알딘과 대립하면서 시작되었다. 1338년(혹은 1339년)에 기야스 알딘이 사망하며 둘 간의 분쟁은 마수드 샤의 승리로 끝을 맺었지만 인주 왕조의 내분은 여기서 그치지 않았다. 샤라프 알딘의 넷째아들 샴스 알딘(Shams al-Din)이 추판 왕조의 피르 호사인(Pir Hosayn)과 힘을 합쳐 시라즈로 쳐들어온 것이었다. 마수드 샤는 북쪽으로 피신했고 샴스 알딘이 인주 왕조의 지배자가 되었다. 하지만 샴스 알딘은 피르 호사인에게 배신당해 살해당했고 피르 호사인 역시 시라즈 내부에서 민심을 잃고 쫓겨났다.\\n\\n피르 호사인은 다음 해에 다시 군대를 이끌고 와서 시라즈를 점령했다. 하지만 이번에도 얼마 지나지 않아 샤라프 알딘의 아들인 아부 이샤크에 의해 쫓겨났다. 아부 이샤크 또한 시라즈를 오래 점유하지 못하고 추판 왕조의 야기 바스티와 힘을 합친 마수드 샤에 의해 쫓겨났다. 하지만 시라즈를 점령한 이후 둘 사이에 알력이 생겼고, 결국 야기 바스티가 마수드 샤를 죽이고 말았다. 마수드 샤를 살해한 야기 바스티 역시 시라즈 내에서의 지지 기반을 잃고 1343년에 아부 이샤크에 의해 쫓겨났다. 아부 이샤크에게 쫓겨난 야기 바스티는 조카인 마렉 아시라프와 힘을 합쳐 시라즈로 다시 쳐들어왔으나 1343년 12월에 추판 왕조의 지배자인 하산 쿠첵이 죽었다는 소식을 듣고 추판 왕조에서의 종주권을 차지하기 위해 본국으로 회군하였다. 이후 아부 이샤크는 1343년부터 인주 왕조가 멸망하는 1357년까지 14년간 인주 왕조의 유일한 지배자로 군림하게 되었다.',
 688,
 '마렉 아시라프')

In [14]:
new_text[new_id:new_id+len(answers[IDX]['text'][0])]

'마렉 아시라프'